# Sequence prediction find most similar

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [ ]:
CONFIG_PATH = '/root/work/artwork_sequence/train_test_configuration/multivariate'
#CONFIG_PATH = '/root/work/artwork_sequence/train_test_configuration/univariate'

In [ ]:
window_index = 3

museum_sequence_path = {
    'x_train' : os.path.join(CONFIG_PATH, 'X_train.csv'),
    'x_test' : os.path.join(CONFIG_PATH, 'X_test.csv'),
    'x_train_matrix' : os.path.join(CONFIG_PATH, 'X_train_matrix.npy'),
    'x_test_matrix' : os.path.join(CONFIG_PATH, 'X_test_matrix.npy'),
    'weights_folder' : os.path.join(CONFIG_PATH, 'config_'+str(window_index)+'/trained_model_weights')
}
museum_sequence_path

## Load data

In [ ]:
df_x_train = pd.read_csv(museum_sequence_path['x_train'], index_col=0)
df_x_test = pd.read_csv(museum_sequence_path['x_test'], index_col=0)
x_train_matrix = np.load(museum_sequence_path['x_train_matrix'])
x_test_matrix = np.load(museum_sequence_path['x_test_matrix'])
df_x_train.head()

In [ ]:
x_train_matrix.shape

## Reset Tensorflow session

In [ ]:
tf.keras.backend.clear_session()

## Config data to fit with the model input

**Define timeline**

In [ ]:
timeline = np.arange(x_test_matrix.shape[0])
timeline.shape

In [ ]:
split_time = x_train_matrix.shape[0]

X = x_test_matrix

window_size = window_index

#Number of feature to take into account
n_influence_features=3

batch_size = 128
shuffle_buffer_size = 300

## Create and Load model

**Get models**

In [ ]:
from Prediction_model_feature import Prediction_model_feature
from Prediction_model_feature_multivariate import Prediction_model_feature_multivariate

In [ ]:
n_features = X.shape[1]
models = []

In [ ]:
from IPython.display import clear_output
import time

start_time = time.time()

for i in range(n_features):
    clear_output(wait=True)
    #Create model
    model_prediction = Prediction_model_feature_multivariate(
        X=X,
        split_time=split_time,
        train_batch_size=batch_size, 
        val_batch_size=batch_size, 
        window_size=window_size, 
        shuffle_buffer=shuffle_buffer_size,
        name="feature " + str(i),
        index = i,
        n_influence_features=n_influence_features,
        n_features=1)
    model_prediction.define_model(conv_filter=32, lstm_filter=64, dense_filter=16)

    #Load weights
    model_prediction.load_weights(museum_sequence_path)

    models.append(model_prediction)
    
print("--- %s seconds ---" % (time.time() - start_time))
len(models)

## Predict tour

In [ ]:
from utils_plot import plot_series, plot_train_history, plot_prediction

**Select a tour**

In [ ]:
df_x_test['tour_path'].values[0]

In [ ]:
images_path = os.path.join(df_x_test['tour_path'].values[0], 'images')
tour_length = len(os.listdir(images_path))
X_tour = x_test_matrix[:tour_length]
X_tour.shape

In [ ]:
metadata_path = os.path.join(df_x_test['tour_path'].values[0], 'metadata.csv')
df_X_tour = pd.read_csv(metadata_path)
df_X_tour.head()

### Predict features artworks for each tours

**Function to predict the tour**

In [ ]:
def model_forecast(model, series, window_size, batch_size):
    if len(series.shape) == 1:
            series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.map(lambda w: (w[:]))
    ds = ds.batch(batch_size)
    forecast = model.predict(ds)
    return forecast

**Predict feature tours**

In [ ]:
df_evaluation = pd.DataFrame({'feature' : [],
                             'forecast': [],
                             'x_valid':[],
                             'mae':[]})
df_evaluation

In [ ]:
for feature in range(n_features):
    #Predict feature i
    #Define feature to take into account for prediction
    x_influence_features = models[feature].get_indexes_features()
    x_influence_features = np.insert(arr=x_influence_features, obj=0, values=feature)
    x_feature = X_tour[:,x_influence_features]
    
    rnn_forecast = model_forecast(models[feature].get_model(), x_feature, window_size, batch_size)
    rnn_forecast = rnn_forecast[1:,-1]

    #Get validation dataset 
    x_valid = x_feature[window_size:, 0]

    #Compute MAE
    mae = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy().mean()

    df_evaluation = df_evaluation.append({'feature' : feature,
                    'forecast': rnn_forecast,
                    'x_valid':x_valid,
                    'mae':mae
                   }, 
                   ignore_index=True)

df_evaluation.head()

In [ ]:
df_evaluation['mae'].mean()

**Plot forecast for a single feature**

In [ ]:
feature = 4
rnn_forecast = df_evaluation.loc[feature,'forecast']
#discard the first artworks
x_valid = df_evaluation.loc[feature,'x_valid']
timeline = np.arange(rnn_forecast.shape[0])

In [ ]:
plot = plot_series(timeline, [(x_valid, 'x_valid'), (rnn_forecast, 'rnn')])
plot.title('Forescast artwork sequence')
#plot.savefig(os.path.join(PLOT_PATH, 'forescast_sequence_2.png'))
plot.show()

## Find most similar artwork

**Load artwork's repository**

In [ ]:
BASE_PATH = '/root/work/datasets/artwork_sequence/'

In [ ]:
df_all_metadata = pd.read_csv(os.path.join(BASE_PATH, 'all_metadata.csv'))
all_data_matrix = np.load(os.path.join(BASE_PATH, 'all_code_matrix.npy' ))

In [ ]:
print(df_all_metadata.shape)
print(all_data_matrix.shape)

**Reconstruct predicted codes**

In [ ]:
forescast = df_evaluation['forecast']
forescast.head()

In [ ]:
forescast[0].shape

In [ ]:
feature_list = list(forescast.values)

In [ ]:
forecast_matrix = np.stack(feature_list)
forecast_matrix = forecast_matrix.T
forecast_matrix.shape

**Helper function to avoid duplicated artworks in a recommendation**

In [ ]:
def drop_selected_artwork(indexes, df_all_metadata, all_data_matrix):
    
    #Remove from metadata
    df_removed = df_all_metadata.copy()
    df_removed = df_removed.drop(indexes)
    df_removed = df_removed.reset_index(drop=True)
    
    #Remove ftom code matrix
    code_matrix = all_data_matrix.copy()
    code_matrix = np.delete(code_matrix, indexes, 0)
    
    return df_removed, code_matrix

**Compute cosine similarity**

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances,euclidean_distances

In [ ]:
predicted_tour_code_list = []
df_predicted_tour = pd.DataFrame({ 'title' : [],
                         'author' : [],
                         'sim_value' : [],
                         'tour_path': [],
                         'image_url':[]})
df_predicted_tour

In [ ]:

for i in range(forecast_matrix.shape[0]):
    #Find code
    code = forecast_matrix[i].reshape((1,-1))
    
    #Compute cosine similarity
    sim_matrix = cosine_similarity(code, all_data_matrix)
    #sim_matrix = euclidean_distances(code, all_data_matrix)
    
    #sort indexes
    sort_index = np.argsort(sim_matrix.reshape((-1,)))
    
    #Find most similar
    sim_artwork_index = sort_index[-1]
    
    #Save in dataframe 
    df_predicted_tour = df_predicted_tour.append({'title' : df_all_metadata.iloc[sim_artwork_index]['title'],
                    'author': df_all_metadata.iloc[sim_artwork_index]['author'],
                    'tour_path':df_all_metadata.iloc[sim_artwork_index]['tour_path'],
                    'image_url':df_all_metadata.iloc[sim_artwork_index]['image_url'],
                    'sim_value':sim_matrix[:,sim_artwork_index][0]
                   }, 
                   ignore_index=True)
    
    #Save predicted artwork's code
    predicted_tour_code_list.append(all_data_matrix[sim_artwork_index])
    
    #Remove selected artworks
    df_all_metadata, all_data_matrix = drop_selected_artwork([sim_artwork_index], df_all_metadata, all_data_matrix)

    

df_predicted_tour

In [ ]:
df_predicted_tour.shape

**Cosine similarity between predicted and true artwork**

In [ ]:
x_valid_artwork = df_evaluation['x_valid']
x_valid_artwork.head()

In [ ]:
artwork_feature_list = list(x_valid_artwork.values)
artwork_matrix = np.stack(artwork_feature_list)
artwork_matrix = artwork_matrix.T
artwork_matrix.shape

In [ ]:
df_sim_true_predicted = pd.DataFrame({ 'title' : [],
                         'author' : [],
                         'sim_value' : [],
                         'tour_path': [],
                         'image_url':[]})
df_sim_true_predicted

In [ ]:
for i in range(forecast_matrix.shape[0]):
    #Find predicted code
    code = forecast_matrix[i].reshape((1,-1))
    #Find true code
    true_artwork_code = artwork_matrix[i].reshape((1,-1))
    
    #Compute cosine similarity
    cos_sim = cosine_similarity(code, true_artwork_code)[0][0]
    #sim_matrix = euclidean_distances(code, all_data_matrix)
        
    #Save in dataframe 
    df_sim_true_predicted= df_sim_true_predicted.append({'title' : df_X_tour.iloc[window_size+i]['title'],
                    'author': df_X_tour.iloc[window_size+i]['author'],
                    'tour_path':df_x_test['tour_path'].values[0],
                    'image_url':df_X_tour.iloc[window_size+i]['image_url'],
                    'sim_value':cos_sim
                   }, 
                   ignore_index=True)
        

df_sim_true_predicted

## Distance between predicted artwork and true artwork

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
columns = ['author', 'title', 'sim_value']

df_compare = pd.concat([df_predicted_tour[columns], df_sim_true_predicted[columns]], axis=1)

df_compare['distance'] = abs(df_predicted_tour['sim_value'] - df_sim_true_predicted['sim_value'])

df_compare

**MAE**

In [ ]:
predicted_artwork_matrix = np.stack(predicted_tour_code_list)
predicted_artwork_matrix = predicted_artwork_matrix.T
predicted_artwork_matrix.shape

In [ ]:
np.mean(mean_absolute_error(predicted_artwork_matrix, artwork_matrix.T, multioutput='raw_values' ))

In [ ]:
np.mean(mean_absolute_error(forecast_matrix.T, artwork_matrix.T, multioutput='raw_values' ))